In [1]:
zimport pandas as pd
import numpy as np
from scipy.stats import norm
from sklearn.metrics import mean_squared_error

/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df1 = pd.read_csv('df1_modified.csv')
df2 = pd.read_csv('df2_modified.csv')
df3 = pd.read_csv('df3_modified.csv')
df_concat = pd.concat([df1, df2, df3], axis=0, ignore_index=True)
df_concat

,underlying_symbol,quote_datetime,root,expiration,strike,option_type,open,high,low,close,...,implied_volatility,delta,gamma,theta,vega,rho,open_interest,risk_free_rate,time_to_maturity,option_price
0,^SPX,2024-07-26 10:30:00,SPX,2024-08-16,200.0,C,0.0,0.0,0.0,0.0,...,5.4324,0.9995,0.0000,0.0,0.0314,-0.0047,24,5.49,0.083333,5250.00
1,^SPX,2024-07-26 11:30:00,SPX,2024-08-16,200.0,C,0.0,0.0,0.0,0.0,...,5.0932,0.9997,0.0000,0.0,0.0202,-0.0117,24,5.49,0.083333,5245.90
2,^SPX,2024-07-26 12:30:00,SPX,2024-08-16,200.0,C,0.0,0.0,0.0,0.0,...,4.9448,0.9998,0.0000,0.0,0.0160,-0.0184,24,5.49,0.083333,5269.65
3,^SPX,2024-07-26 13:30:00,SPX,2024-08-16,200.0,C,0.0,0.0,0.0,0.0,...,4.8580,1.0000,0.0000,0.0,0.0141,-0.0210,24,5.49,0.083333,5284.75
4,^SPX,2024-07-26 14:30:00,SPX,2024-08-16,200.0,C,0.0,0.0,0.0,0.0,...,4.2050,1.0000,0.0000,0.0,0.0034,-0.0269,24,5.49,0.083333,5258.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513725,^SPX,2024-09-27 12:30:00,SPXW,2025-09-30,7800.0,P,0.0,0.0,0.0,0.0,...,0.1322,-0.9816,0.0001,0.0,3.6619,-74.5091,2,3.55,1.460317,1794.85
513726,^SPX,2024-09-27 13:30:00,SPXW,2025-09-30,7800.0,P,0.0,0.0,0.0,0.0,...,0.1304,-0.9837,0.0001,0.0,3.4177,-74.6205,2,3.55,1.460317,1799.45
513727,^SPX,2024-09-27 14:30:00,SPXW,2025-09-30,7800.0,P,0.0,0.0,0.0,0.0,...,0.1302,-0.9844,0.0001,0.0,3.3423,-74.6503,2,3.55,1.460317,1804.95
513728,^SPX,2024-09-27 15:30:00,SPXW,2025-09-30,7800.0,P,0.0,0.0,0.0,0.0,...,0.1301,-0.9837,0.0001,0.0,3.4058,-74.6146,2,3.55,1.460317,1797.75


In [3]:
# Black-Scholes model for Call and Put options
N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    if K == 0 or T == 0 or sigma == 0:
        return np.nan  # Return NaN for invalid values
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T) * N(d2)

def BS_PUT(S, K, T, r, sigma):
    if K == 0 or T == 0 or sigma == 0:
        return np.nan  # Return NaN for invalid values
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r*T) * N(-d2) - S * N(-d1)

# Apply the Black-Scholes formula to calculate option prices for each row in the dataset
def calculate_option_price(row):
    S = row['active_underlying_price']
    K = row['strike']
    T = row['time_to_maturity']
    r = row['risk_free_rate']/100  # Ensure risk-free rate is in decimal form (not percentage)
    sigma = row['implied_volatility'] / 100  # Ensure volatility is in decimal form (not percentage)

    if row['option_type'] == 'C':  # Call option
        return BS_CALL(S, K, T, r, sigma)
    elif row['option_type'] == 'P':  # Put option
        return BS_PUT(S, K, T, r, sigma)

# Calculate the option price and add it to a new column in the dataframe
df_concat['BS_calculated_option_price'] = df_concat.apply(calculate_option_price, axis=1)

In [4]:
df_concat.to_csv('df_concat_withBSMprice.csv', index=False)

In [5]:

# Filter rows where neither BS_calculated_option_price nor option_price is NaN, and the calculated price is not 0
valid_df = df_concat.dropna(subset=['BS_calculated_option_price', 'option_price'])
valid_df = valid_df[valid_df['BS_calculated_option_price'] != 0]

# Get the actual option price and the calculated BS option price
actual_prices = valid_df['option_price']
calculated_prices = valid_df['BS_calculated_option_price']

# Calculate Mean Squared Error (MSE)
mse = mean_squared_error(actual_prices, calculated_prices)

# Calculate Root Mean Squared Error (RMSE)
rmse = np.sqrt(mse)

mse, rmse


(12667.726913250768, 112.55099694472176)